In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def load_az_dataset(datasetPath):
    # initialize the list of data and labels
    data = []
    labels = []
    # loop over the rows of the A-Z handwritten digit dataset
    for row in open(datasetPath):
        # parse the label and image from the row
        row = row.split(",")
        label = int(row[0])
        image = np.array([int(x) for x in row[1:]], dtype="uint8")
        # images are represented as single channel (grayscale) images
        # that are 28x28=784 pixels -- we need to take this flattened
        # 784-d list of numbers and repshape them into a 28x28 matrix
        image = image.reshape((28, 28))
        # update the list of data and labels
        data.append(image)
        labels.append(label)
    # convert the data and labels to NumPy arrays
    data = np.array(data, dtype="float32")
    labels = np.array(labels, dtype="int")
    # return a 2-tuple of the A-Z data and labels
    return (data, labels)

In [3]:
def load_mnist_dataset():
    # load the MNIST dataset and stack the training data and testing
    # data together (we'll create our own training and testing splits
    # later in the project)
    ((trainData, trainLabels), (testData, testLabels)) = tf.keras.datasets.mnist.load_data()
    data = np.vstack([trainData, testData])
    labels = np.hstack([trainLabels, testLabels])
    # return a 2-tuple of the MNIST data and labels
    return (data, labels)

In [4]:
(azData, azLabels) = load_az_dataset('NIST_SD19/A_Z Handwritten Data/A_Z Handwritten Data.csv')
(digitsData, digitsLabels) = load_mnist_dataset()

In [5]:
azLabels += 10
data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])

In [6]:
# convert the labels from integers to vectors
le = LabelBinarizer()
labels = le.fit_transform(labels)
counts = labels.sum(axis=0)
# account for skew in the labeled data
classTotals = labels.sum(axis=0)
classWeight = {}
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
	classWeight[i] = classTotals.max() / classTotals[i]

In [7]:
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=42)

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(36, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 36)                4644      
                                                                 
Total params: 105,124
Trainable params: 105,124
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50)

Epoch 1/50
8849/8849 [==============================] - 31s 3ms/step - loss: 1.7123 - accuracy: 0.6752 - val_loss: 0.8356 - val_accuracy: 0.7884
Epoch 2/50
8849/8849 [==============================] - 32s 4ms/step - loss: 0.7872 - accuracy: 0.8024 - val_loss: 0.7441 - val_accuracy: 0.8102
Epoch 3/50
8849/8849 [==============================] - 32s 4ms/step - loss: 0.7342 - accuracy: 0.8176 - val_loss: 0.7527 - val_accuracy: 0.8139
Epoch 4/50
8849/8849 [==============================] - 33s 4ms/step - loss: 0.7137 - accuracy: 0.8246 - val_loss: 0.7027 - val_accuracy: 0.8259
Epoch 5/50
8849/8849 [==============================] - 32s 4ms/step - loss: 0.7083 - accuracy: 0.8284 - val_loss: 0.7101 - val_accuracy: 0.8252
Epoch 6/50
8849/8849 [==============================] - 32s 4ms/step - loss: 0.7051 - accuracy: 0.8307 - val_loss: 0.7360 - val_accuracy: 0.8276
Epoch 7/50
8849/8849 [==============================] - 33s 4ms/step - loss: 0.7024 - accuracy: 0.8317 - val_loss: 0.7346 - val_ac

In [10]:
# optional model load
model = tf.keras.models.load_model('model/basic.h5')

In [14]:
predictions = model.predict(X_test[:3000])
print(classification_report(y_test[:3000].argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

              precision    recall  f1-score   support

           0       1.00      0.57      0.73        40
           1       0.93      0.83      0.88        64
           2       0.85      0.69      0.76        48
           3       0.80      0.65      0.72        43
           4       0.76      0.52      0.62        50
           5       0.94      0.69      0.79        45
           6       1.00      0.85      0.92        46
           7       0.91      0.87      0.89        46
           8       0.67      0.59      0.63        49
           9       0.82      0.93      0.87        40
           A       0.86      0.86      0.86        90
           B       0.65      0.55      0.60        62
           C       0.90      0.93      0.92       143
           D       0.76      0.83      0.79        72
           E       0.75      0.88      0.81        88
           F       1.00      0.50      0.67         4
           G       0.59      0.47      0.52        34
           H       0.59    

C:\Users\monol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\monol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\monol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
predictions.shape

(3000, 36)

In [12]:
import gradio as gr

def classify(input):
    prediction = model.predict(input.reshape(1, 28, 28)).tolist()[0]
    return {str(label): prediction[i] for i, label in enumerate(labelNames)}

label = gr.outputs.Label(num_top_classes=3)
interface = gr.Interface(fn=classify, inputs="sketchpad", outputs=label)
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861/
Running on public URL: https://57809.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://57809.gradio.app')

In [15]:
model.save('model/Basic_Model.model')

In [18]:
import pickle

with open('hist/Basic_Model.hist', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)